# Yelp

**[Work in progress]**

This notebook creates a .csv file with yelp information for ingestion into the Knowledge Graph.

In [55]:
import os
import json
import pandas as pd 

from pathlib import Path
from yelpapi import YelpAPI
from pprint import pprint


In [21]:
# insert rest api key here
api_key = ''
yelp_api = YelpAPI(api_key)

In [47]:
# San Diego
term = 'restaurant'
latitude = 32.710
longitude = -117.170
limit = 50
radius = 5000 # 5km radius = 0.1 degree
offset = 0

In [90]:
# Maximum 1000 restaurant can be pulled
scrapped = 0
df_combined = None 
for i in range(20):
    print('Scrapping offset: {}'.format(50 * i))
    response = yelp_api.search_query(term=term, latitude=latitude, longitude=longitude, 
                                     limit=limit, radius=radius, offset=scrapped)
    scrapped += len(response['businesses'])
    df = pd.json_normalize(response['businesses'])    
    
    if df_combined is not None:
        df_combined = pd.concat([df_combined, df], sort=False, ignore_index=True)
    else:       
        df_combined = df.copy()

    # More to scrapped from this location
    if response['total'] <= scrapped:
        print('Completed scrapping for this location')
        break

Scrapping offset: 0
Scrapping offset: 50
Scrapping offset: 100
Scrapping offset: 150
Scrapping offset: 200
Scrapping offset: 250
Scrapping offset: 300
Scrapping offset: 350
Scrapping offset: 400
Scrapping offset: 450
Scrapping offset: 500
Scrapping offset: 550
Scrapping offset: 600
Scrapping offset: 650
Scrapping offset: 700
Scrapping offset: 750
Scrapping offset: 800
Scrapping offset: 850
Scrapping offset: 900
Scrapping offset: 950


In [91]:
response['total']

1400

In [92]:
columns={
    'coordinates.latitude':'coordinates_latitude',
    'coordinates.longitude':'coordinates_longitude',    
    'location.address1':'location_address1',
    'location.address2':'location_address2',
    'location.address3':'location_address3',
    'location.city':'location_city',
    'location.zip_code':'location_zip_code',
    'location.country':'location_country',
    'location.state':'location_state',
    'location.display_address':'location_display_address',
}
df_final = df_combined.rename(columns=columns)

In [93]:
df_final.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates_latitude,coordinates_longitude,location_address1,location_address2,location_address3,location_city,location_zip_code,location_country,location_state,location_display_address
0,4_vPdLbDx0Iadr2nANyCQQ,puesto-at-the-headquarters-san-diego-7,Puesto at the Headquarters,https://s3-media2.fl.yelpcdn.com/bphoto/ixmqgQ...,False,https://www.yelp.com/biz/puesto-at-the-headqua...,3862,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"[pickup, delivery]",...,32.711340,-117.170000,789 W Harbor Dr,Ste 155,None,San Diego,92101,US,CA,"[789 W Harbor Dr, Ste 155, San Diego, CA 92101]"
1,Sa_qcnc7ZgzSOylf3plTRA,c-level-san-diego,C Level,https://s3-media4.fl.yelpcdn.com/bphoto/rE2Ti2...,False,https://www.yelp.com/biz/c-level-san-diego?adj...,3159,"[{'alias': 'newamerican', 'title': 'American (...",4.0,[],...,32.724183,-117.188348,880 Harbor Island Dr,,,San Diego,92101,US,CA,"[880 Harbor Island Dr, San Diego, CA 92101]"
2,OW2VM6KY0s2Ir04xg_o1sg,eddie-vs-prime-seafood-and-steaks-san-diego-2,Eddie V's Prime Seafood and Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/lYxMnk...,False,https://www.yelp.com/biz/eddie-vs-prime-seafoo...,1346,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,[],...,32.710496,-117.170761,789 W Harbor Dr,,,San Diego,92101,US,CA,"[789 W Harbor Dr, San Diego, CA 92101]"
3,7g8CKU-UqEJ4bnIH2G5HCQ,cafe-sevilla-san-diego,Cafe Sevilla,https://s3-media3.fl.yelpcdn.com/bphoto/9HZ24s...,False,https://www.yelp.com/biz/cafe-sevilla-san-dieg...,3331,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.0,"[restaurant_reservation, pickup, delivery]",...,32.708977,-117.159964,353 5th Ave,,,San Diego,92101,US,CA,"[353 5th Ave, San Diego, CA 92101]"
4,W0dzJ7E6R3kyAOXIlO_tbw,ristorante-illando-san-diego,Ristorante Illando,https://s3-media3.fl.yelpcdn.com/bphoto/FXO-Kk...,False,https://www.yelp.com/biz/ristorante-illando-sa...,1204,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"[pickup, restaurant_reservation, delivery]",...,32.724049,-117.168190,1825 India Street,None,,San Diego,92101,US,CA,"[1825 India Street, San Diego, CA 92101]"


In [94]:
len(df_final)

1000

In [95]:
with open('./../data/yelp.csv', 'w') as f:
    df_final.to_csv(f, index=False)